In [1]:
import pandas as pd
import numpy as np
import sys

### Import Oracle Creator

In [2]:
sys.path.append("../../../../Tracer/src")
from DataHandling.TraceCreator import create_trace_matrix_from_trace_list

def create_trace_list(path_to_oracle):
    oracle = open(path_to_oracle).read().split("\n")
    trace_list = []
    for trace_item in oracle:
        if len(trace_item) > 0:
            source, targets = trace_item.split(":")
            for target in targets.split(" "):
                if len(target) != 0:
                    trace_list.append(" ".join([source, target]))
                else:
                    trace_list.append(" ".join([source, ""]))
    return create_trace_matrix_from_trace_list(trace_list)

# Original Oracles

In [3]:
r2d = pd.read_csv("../../Oracles/DataFrames/Level_1_to_Level_2.csv").set_index("id")

### Import New Oracle

In [4]:
r2r = create_trace_list("../Originals/TracedMatrices/UC_UC.txt")
r2r.head(3)

,17.txt,25.txt,29.txt,19.txt,18.txt,3.txt,28.txt,9.txt,2.txt,8.txt,...,1.txt,6.txt,27.txt,16.txt,12.txt,20.txt,11.txt,23.txt,4.txt,30.txt
id,,,,,,,,,,,,,,,,,,,,,
17.txt,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
25.txt,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
19.txt,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


### R 2 D

In [5]:
def get_similar_requirements(r_id):
    similar_requirements = []
    if r_id in r2r.index:
        for r2_id in r2r.columns:
            if r2r.loc[r_id][r2_id] == 1:
                similar_requirements.append(r2_id)
    return similar_requirements

In [14]:
missing_traces = []
n_missing_traces = 0
n_confirmed_traces = 0
n_undefined_classes = 0
for r_id in r2d.index:
    for d_id in r2d.columns:
        if r2d.loc[r_id][d_id] == 1: # for each pair of similar requirements
            similar_requirements = get_similar_requirements(r_id)
            for s_req in similar_requirements:
                if s_req not in r2d.index:
                    n_undefined_classes = n_undefined_classes + 1
                elif s_req in r2d.index and r2d.loc[s_req][d_id] == 0:
                    n_missing_traces = n_missing_traces + 1
                    missing_traces.append((s_req, d_id))
                elif s_req in r2d.index and r2d.loc[s_req][d_id] == 1:
                    n_confirmed_traces = n_confirmed_traces + 1
print("Number of Missing Traces: ", len(missing_traces))
print("NUmber of confirmed traces: ", n_confirmed_traces)
print("Number of undefined classes: ", n_undefined_classes)

Number of Missing Traces:  0
NUmber of confirmed traces:  806
Number of undefined classes:  0


# Add missing traces

In [15]:
for mt in missing_traces:
    r2d.loc[mt[0]][mt[1]] = 1

In [16]:
r2d.to_csv("../../Oracles/DataFrames/Level_1_to_Level_2.csv")
#r2c.to_csv("../../Oracles/Relations.csv")
print("Done!")

Done!
